In [41]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [42]:
################################
# Imports
################################

import pandas as pd
import h5py
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import glob
import re
from multiprocessing import cpu_count
from multiprocessing import Pool

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import os
import sys
import subprocess
import random
from random import sample
from tqdm import tqdm
from functools import partial

from sklearn.metrics import confusion_matrix


import librosa

In [43]:
################################
# configs
################################

#################
# spectrogram
#################

# Mean of flute = -6.438913345336914
# Median of flute = -6.118330955505371
# stdDev of flute = 4.377377986907959
# Max of flute = 1.8442230224609375
# Min of flute = -39.0754280090332

# Mean of piano = -6.015857219696045
# Median of piano = -5.299488544464111
# stdDev of piano = 4.420877456665039
# Max of piano = 1.5825170278549194
# Min of piano = -40.520179748535156

spectrogramStats = {
                    'flute': {'mean': -6.438913345336914, 'median': -6.118330955505371, 'stdDev': 4.377377986907959, 'max': 1.8442230224609375, 'min': -39.0754280090332},
                    'piano': {'mean': -6.015857219696045, 'median': -5.299488544464111, 'stdDev': 4.420877456665039, 'max': 1.5825170278549194, 'min': -40.520179748535156}
                    }

# standardizationStyleOptions = normal, logNormal, uniform
# following needs to be 'SUBTRACTED' from the data
centerOffset = {
                'flute': {'normal': spectrogramStats['flute']['mean'], 'logNormal': spectrogramStats['flute']['mean'], 'uniform': (spectrogramStats['flute']['min'] + spectrogramStats['flute']['max'])/2},
                'piano': {'normal': spectrogramStats['piano']['mean'], 'logNormal': spectrogramStats['piano']['mean'], 'uniform': (spectrogramStats['piano']['min'] + spectrogramStats['piano']['max'])/2}
                }

# following needs to be 'DIVIDED' to the data
divFactor = {
            'flute': {'normal': 3*spectrogramStats['flute']['stdDev'], 'logNormal': (1.1*spectrogramStats['flute']['max'] - spectrogramStats['flute']['mean']) , 'uniform': 1*(spectrogramStats['flute']['max'] - spectrogramStats['flute']['min'])/2},
            'piano': {'normal': 3*spectrogramStats['piano']['stdDev'], 'logNormal': (1.1*spectrogramStats['piano']['max'] - spectrogramStats['piano']['mean']) , 'uniform': 1*(spectrogramStats['piano']['max'] - spectrogramStats['piano']['min'])/2}
            }


####################
# training params
####################
STANDARDIZATION_STYLE = 'uniform'
BATCH_SIZE = 4
NUM_WORKERS = 2
NUM_EPOCHS = 10
LEARNING_RATE = 0.0001
LAMBDA_CYCLE = 10
LAMBDA_IDENTITY = 5
GRADIENT_PENALTY = 0
ADAM_BETA1 = 0
ADAM_BETA2 = 0.9
NUM_RESIDUALS = 9

####################
# checkpoint options
####################
SAVE_CHECKPOINTS = True

####################
# find GPU device
####################
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (f'Device = {DEVICE}, # of CUDA devices = {torch.cuda.device_count()}')


###################
# add fileTag
###################
fileTag = f'lr_{LEARNING_RATE}_cyc_{LAMBDA_CYCLE}_id_{LAMBDA_IDENTITY}_b1_{ADAM_BETA1}_b2_{ADAM_BETA2}_numRes_{NUM_RESIDUALS}_gp_{GRADIENT_PENALTY}_stdStyle_{STANDARDIZATION_STYLE}'


Device = cuda:0, # of CUDA devices = 1


In [44]:
################################
# discriminator
################################

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 4, stride = stride, padding = 1, bias=True, padding_mode="reflect"),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=1, features=[64, 128, 256, 512], numFlatFeatures = 1200):
        
        super().__init__()
        
        # output shape = 168 x 128
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels, features[0], kernel_size = 4, stride = 2, padding = 1, bias=True, padding_mode="reflect"),            
            nn.LeakyReLU(0.2)
        )
        
        
        layers = []   
        in_channels = features[0]
        
        # linear layer will have 20 x 15 features flattened
        for feature in features[1:]:
            layers.append(Block(in_channels, feature, stride=1 if feature == features[-1] else 2))
            in_channels = feature        
        
        layers.append(nn.Sequential(
            nn.Conv2d(in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"), 
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(in_features = numFlatFeatures, out_features = 1)
        ))
        
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):        
        x = self.initial(x)
        return torch.sigmoid(self.model(x))
        
    
def test():
    x = torch.randn((5,1,336,256))
    model = Discriminator(in_channels=1)
    print (model)
    preds = model(x)
    print(preds.shape)

test()

Discriminator(
  (initial): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (model): Sequential(
    (0): Block(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
    (1): Block(
      (conv): Sequential(
        (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
    (2): Block(
      (conv): Sequential(
        (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(512, eps=1e-05, momentum

In [45]:
################################
# generator
################################

class EncoderConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, use_act=True, use_tanh = False, **kwargs):
        super().__init__()
        self.encoderConv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs),
            nn.InstanceNorm2d(out_channels),
            (nn.Tanh() if use_tanh else nn.ReLU(inplace=True)) if use_act else nn.Identity()
        )

    def forward(self, x):
        return self.encoderConv(x)


class DecoderConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, use_act=True, use_tanh = False):
        super().__init__()
        
        # nearest neighbor upsample + same convolution
        self.decoderConv = nn.Sequential(            
            nn.Upsample(scale_factor = 2, mode='nearest'),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=0),
            nn.InstanceNorm2d(out_channels),
            (nn.Tanh() if use_tanh else nn.ReLU(inplace=True)) if use_act else nn.Identity()
        )

    def forward(self, x):
        return self.decoderConv(x)
    

class ResidualBlock(nn.Module):
    def __init__(self, channels, use_tanh = False):
        super().__init__()
        self.block = nn.Sequential(
            EncoderConvBlock(channels, channels, use_act = True,  use_tanh = use_tanh, kernel_size=3, stride = 1, padding=1),
            EncoderConvBlock(channels, channels, use_act = False, use_tanh = use_tanh, kernel_size=3, stride = 1, padding=1)
        )

    def forward(self, x):
        return x + self.block(x)


class Generator(nn.Module):
    
    def __init__(self, img_channels, use_tanh = False, num_features = 64, num_residuals=9):
        
        super().__init__()
        
        # same convolution. output channels = 16
        self.initial = nn.Sequential(            
            nn.Conv2d(img_channels, num_features, kernel_size=7, stride=1, padding=3, padding_mode="reflect"),
            nn.ReLU(inplace=True)
        )
        
        
        self.down_blocks = nn.ModuleList(
            [
                # output shape = floor((W - F + 2P) / S) + 1 . 
                # Wout x Hout = 128 x 168, for Win x Hin = 256 x 336, output channels = 32
                EncoderConvBlock(num_features, num_features*2, use_act = True, use_tanh = use_tanh, kernel_size=3, stride=2, padding=1),
                
                # Wout x Hout = 64 x 84, for Win x Hin = 128 x 168, output channels = 64
                EncoderConvBlock(num_features*2, num_features*4, use_act = True, use_tanh = use_tanh, kernel_size=3, stride=2, padding=1),
            ]
        )

        # same convolutions in residual blocks. Shape remains 64 x 86, numChannels = 64
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(num_features*4, use_tanh = use_tanh) for _ in range(num_residuals)]
        )

        self.up_blocks = nn.ModuleList(
            [
                # Wout x Hout = 128 x 168, numChannels = 32
                DecoderConvBlock(num_features*4, num_features*2, use_act = True, use_tanh = use_tanh),
                
                # Wout x Hout = 256 x 336, numChannels = 16
                DecoderConvBlock(num_features*2, num_features, use_act = True, use_tanh = use_tanh)
            ]
        )

        # same convolution, numChannels = 1
        self.last = nn.Conv2d(num_features, img_channels, kernel_size=7, stride=1, padding=3, padding_mode="reflect")

    def forward(self, x):
        
        # same convolution
        x = self.initial(x)
        
        # down sampling
        for layer in self.down_blocks:
            x = layer(x)
            
        # same convolutions
        x = self.residual_blocks(x)
        
        # upsampling
        for layer in self.up_blocks:
            x = layer(x)
        
        # same convolution
        x = self.last(x)
        
        return torch.tanh(x)

    
def test():
    x = torch.randn((5,1,336,256))
    model = Generator(img_channels=1, num_features = 16, num_residuals=9)
    print (model)
    gen = model(x)    
    print(gen.shape)

    
test()

Generator(
  (initial): Sequential(
    (0): Conv2d(1, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), padding_mode=reflect)
    (1): ReLU(inplace=True)
  )
  (down_blocks): ModuleList(
    (0): EncoderConvBlock(
      (encoderConv): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
    (1): EncoderConvBlock(
      (encoderConv): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
  )
  (residual_blocks): Sequential(
    (0): ResidualBlock(
      (block): Sequential(
        (0): EncoderConvBlock(
          (encoderConv): Sequential(
            (0): Conv2d(64, 64, kernel_si

torch.Size([5, 1, 336, 256])


In [46]:
# data loader

class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, rootDir):
        'Initialization'
        self.rootDir = rootDir
        self.fileList = os.listdir(rootDir)        
        

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.fileList)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        X = np.load(os.path.join(self.rootDir, self.fileList[index]))
        
        return X

In [47]:
##########################
# dataset class
##########################

class PianoFluteDataset(Dataset):
    def __init__(self, root_piano, root_flute, standardizationStyle):
        self.root_piano = root_piano
        self.root_flute = root_flute

        random.seed(2)
        self.piano_images = os.listdir(root_piano)
        self.flute_images = sample(os.listdir(root_flute), len(os.listdir(root_flute)))
        
        self.length_dataset = max(len(self.piano_images), len(self.flute_images))

        self.piano_dataset_length = len(self.piano_images)
        self.flute_dataset_length = len(self.flute_images)
        
        self.standardizationStyle = standardizationStyle
        
        # default to normal for standardizing the data
        self.offsetFlute = centerOffset['flute']['normal']
        self.offsetPiano = centerOffset['piano']['normal']
        
        self.divFlute = divFactor['flute']['normal']
        self.divPiano = divFactor['piano']['normal']
        
        if standardizationStyle == 'uniform':
            print ('Using uniform assumption...')
            self.offsetFlute = centerOffset['flute']['uniform']
            self.offsetPiano = centerOffset['piano']['uniform']
        
            self.divFlute = divFactor['flute']['uniform']
            self.divPiano = divFactor['piano']['uniform']
            
        else:
            print ('Using logNormal assumption...')
            self.offsetFlute = centerOffset['flute']['logNormal']
            self.offsetPiano = centerOffset['piano']['logNormal']
        
            self.divFlute = divFactor['flute']['logNormal']
            self.divPiano = divFactor['piano']['logNormal']
        

    def __len__(self):
        return self.length_dataset

    def __getitem__(self, index):
        
        flute_image = self.flute_images[index % self.flute_dataset_length]
        piano_image = self.piano_images[index % self.piano_dataset_length]

        flute_path = os.path.join(self.root_flute, flute_image)
        piano_path = os.path.join(self.root_piano, piano_image)
        
        flute_img = np.load(flute_path)
        piano_img = np.load(piano_path)
        
        # add extra dimension for the single channel
        flute_img = flute_img[None, :]
        piano_img = piano_img[None, :]
        
        # standardize the scale
        flute_img = (flute_img - self.offsetFlute) / self.divFlute
        piano_img = (piano_img - self.offsetPiano) / self.divPiano        

        return flute_img, piano_img

In [48]:
def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):    
    checkpoint = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)

In [49]:
################################
# paths to directories
################################

curDir = os.getcwd()
trainSetDir = f'{curDir}/../../dataSuperSet/processedData/trainSet'
fluteTrainSetDir = f'{trainSetDir}/flute/cqtChunks'
pianoTrainSetDir = f'{trainSetDir}/piano/cqtChunks'

# create directories to store checkpoint outputs
checkPointDir = f'{curDir}/checkPoints'
checkPointModelDir = f'{checkPointDir}/models'
checkPointImageDir = f'{checkPointDir}/images'
checkPointLossTrackingDir = f'{checkPointDir}/lossTracking'

os.system(f'mkdir -p {checkPointDir}')
os.system(f'mkdir -p {checkPointModelDir}')
os.system(f'mkdir -p {checkPointImageDir}')
os.system(f'mkdir -p {checkPointLossTrackingDir}')

0

In [50]:
################################
# datasets
################################

# create dataset
dataset = PianoFluteDataset(pianoTrainSetDir, fluteTrainSetDir, STANDARDIZATION_STYLE)

# create dataloader
loader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

Using uniform assumption...


In [51]:
#########################################
# instantiate networks and optimizers
#########################################

# discriminator piano
disc_P = Discriminator(in_channels=1).to(DEVICE)

# discriminator flute
disc_F = Discriminator(in_channels=1).to(DEVICE)

# generator piano
gen_P = Generator(img_channels=1, num_residuals = NUM_RESIDUALS).to(DEVICE)

# generator piano
gen_F = Generator(img_channels=1, num_residuals = NUM_RESIDUALS).to(DEVICE)

# optimizer discriminator
opt_disc = optim.Adam(list(disc_P.parameters()) + list(disc_F.parameters()), lr = LEARNING_RATE, betas=(ADAM_BETA1, ADAM_BETA2))

# optimizer generator
opt_gen  = optim.Adam(list(gen_P.parameters())  + list(gen_F.parameters()),  lr = LEARNING_RATE, betas=(ADAM_BETA1, ADAM_BETA2))


# Losses

# For cycle consistency and identity loss
L1 = nn.L1Loss() 

# adversarial loss
mse = nn.MSELoss()


In [52]:
################################
# training
################################

generatorLossProgression = np.zeros(NUM_EPOCHS)
discriminatorLossProgression = np.zeros(NUM_EPOCHS)
identityLossProgression = np.zeros(NUM_EPOCHS)
cycleLossProgression = np.zeros(NUM_EPOCHS)

for epoch in range(NUM_EPOCHS):    
    
    for idx, (flute, piano) in enumerate(loader):
        
        # move data to device
        piano = piano.to(DEVICE)
        flute = flute.to(DEVICE)
                
        
        ##############################
        # Discriminator training
        ##############################
        
        # piano generator output 
        fake_piano = gen_P (flute)        
        
        # piano discriminator
        D_P_real = disc_P(piano)
        D_P_fake = disc_P(fake_piano.detach())                
        
        # real = 1, fake = 0
        D_P_real_loss = mse(D_P_real, torch.ones_like(D_P_real))
        D_P_fake_loss = mse(D_P_fake, torch.zeros_like(D_P_real))
                
        
        # total piano discriminator loss
        Disc_piano_loss = D_P_real_loss + D_P_fake_loss

        # flute generator output
        fake_flute = gen_F(piano)
        
        # flute discriminator
        D_F_real = disc_F(flute)
        D_F_fake = disc_F(fake_flute.detach())
        
        # real = 1, fake = 0
        D_F_real_loss = mse(D_F_real, torch.ones_like(D_F_real))
        D_F_fake_loss = mse(D_F_fake, torch.zeros_like(D_F_real))
        
        # total flute discriminator loss
        Disc_flute_loss = D_F_real_loss + D_F_fake_loss

        # Overall discriminator loss
        D_loss = (Disc_flute_loss + Disc_piano_loss)/2

        # zero out the gradients
        opt_disc.zero_grad()
        
        # backprop
        D_loss.backward()
        
        # update discriminator params
        opt_disc.step()        
        
        ##############################
        # Generator training
        ##############################
        
        # Adversarial Loss for both generators
        D_P_fake = disc_P(fake_piano)
        D_F_fake = disc_F(fake_flute)
        
        # generator wants fake to be detected real
        loss_G_F = mse(D_F_fake, torch.ones_like(D_F_fake))
        loss_G_P = mse(D_P_fake, torch.ones_like(D_P_fake))

        # Cycle Loss
        cycle_piano = gen_P(fake_flute)
        cycle_flute = gen_F(fake_piano)
        cycle_piano_loss = L1(piano, cycle_piano)
        cycle_flute_loss = L1(flute, cycle_flute)
        
        cycle_loss = cycle_flute_loss + cycle_piano_loss

        # Identity Loss
        identity_flute = gen_F(flute)
        identity_piano = gen_P(piano)
        identity_piano_loss = L1(piano, identity_piano)
        identity_flute_loss = L1(flute, identity_flute)
        
        identity_loss = identity_flute_loss + identity_piano_loss

        # Overall Generator Loss
        G_loss = loss_G_F + loss_G_P + cycle_loss*LAMBDA_CYCLE + identity_loss*LAMBDA_IDENTITY

        # zero out the gradients
        opt_gen.zero_grad()
        
        # backprop
        G_loss.backward()
        
        # update generator params
        opt_gen.step()
        
        if idx % 100 == 0:
            
            # print the current losses
            print (f'[epoch = {epoch}, idx = {idx}]:   D_loss = {D_loss.item()}, \t G_loss = {G_loss.item()}, \t identity_loss = {identity_loss.item()}, \t cycle_loss = {cycle_loss.item()}')            
            
        if idx % 1250 == 0:      
            
            # save the fake piano and flute np array of a sample            
            np.save(f'{checkPointImageDir}/originalPiano__idx_{idx}__{fileTag}.npy', np.squeeze(piano.detach().cpu().numpy()[0,:]))
            np.save(f'{checkPointImageDir}/fakeFlute__idx_{idx}__{fileTag}.npy', np.squeeze(fake_flute.detach().cpu().numpy()[0,:]))            
            np.save(f'{checkPointImageDir}/originalFlute__idx_{idx}__{fileTag}.npy', np.squeeze(flute.detach().cpu().numpy()[0,:]))
            np.save(f'{checkPointImageDir}/fakePiano__idx_{idx}__{fileTag}.npy', np.squeeze(fake_piano.detach().cpu().numpy()[0,:]))
        
        
        # aggregate loss over all batches
        discriminatorLossProgression[epoch] += D_loss.item()
        generatorLossProgression[epoch] += G_loss.item()
        identityLossProgression[epoch] += identity_loss.item()
        cycleLossProgression[epoch] += cycle_loss.item()
                
    
    # average loss over the length of dataset
    discriminatorLossProgression[epoch] = discriminatorLossProgression[epoch] / len(dataset)
    generatorLossProgression[epoch] = generatorLossProgression[epoch] / len(dataset)
    identityLossProgression[epoch] = identityLossProgression[epoch] / len(dataset)
    cycleLossProgression[epoch] = cycleLossProgression[epoch] / len(dataset)
    
    # save the model and current losses
    if SAVE_CHECKPOINTS:
        save_checkpoint(gen_P, opt_gen, filename = f'{checkPointModelDir}/genp__{fileTag}.pth.tar')
        save_checkpoint(gen_F, opt_gen, filename = f'{checkPointModelDir}/genf__{fileTag}.pth.tar')
        save_checkpoint(disc_P, opt_disc, filename = f'{checkPointModelDir}/criticp__{fileTag}.pth.tar')
        save_checkpoint(disc_F, opt_disc, filename = f'{checkPointModelDir}/criticf__{fileTag}.pth.tar')
    
        # save the lossProgressions
        np.save(f'{checkPointLossTrackingDir}/discriminatorLossProgression__{fileTag}.npy', discriminatorLossProgression)
        np.save(f'{checkPointLossTrackingDir}/generatorLossProgression__{fileTag}.npy', generatorLossProgression)
        np.save(f'{checkPointLossTrackingDir}/identityLossProgression__{fileTag}.npy', identityLossProgression)
        np.save(f'{checkPointLossTrackingDir}/cycleLossProgression__{fileTag}.npy', cycleLossProgression)
    





[epoch = 0, idx = 0]:   D_loss = 0.5185933709144592, 	 G_loss = 24.398632049560547, 	 identity_loss = 1.556258201599121, 	 cycle_loss = 1.5451085567474365
[epoch = 0, idx = 100]:   D_loss = 0.03963799029588699, 	 G_loss = 8.744550704956055, 	 identity_loss = 0.43004947900772095, 	 cycle_loss = 0.46273547410964966
[epoch = 0, idx = 200]:   D_loss = 0.0002331225259695202, 	 G_loss = 6.771961212158203, 	 identity_loss = 0.3030032515525818, 	 cycle_loss = 0.3286404311656952
[epoch = 0, idx = 300]:   D_loss = 0.4556547701358795, 	 G_loss = 5.681748867034912, 	 identity_loss = 0.22740718722343445, 	 cycle_loss = 0.2559446096420288
[epoch = 0, idx = 400]:   D_loss = 1.9122674075333634e-06, 	 G_loss = 3.502599000930786, 	 identity_loss = 0.09092985093593597, 	 cycle_loss = 0.10484661906957626
[epoch = 0, idx = 500]:   D_loss = 1.4410979019885417e-05, 	 G_loss = 3.7300760746002197, 	 identity_loss = 0.11086978018283844, 	 cycle_loss = 0.11820048093795776
[epoch = 0, idx = 600]:   D_loss = 0.210

[epoch = 0, idx = 5100]:   D_loss = 0.0002798505302052945, 	 G_loss = 2.602221727371216, 	 identity_loss = 0.03748704493045807, 	 cycle_loss = 0.04150018095970154
[epoch = 0, idx = 5200]:   D_loss = 0.14006999135017395, 	 G_loss = 3.0124025344848633, 	 identity_loss = 0.07636044174432755, 	 cycle_loss = 0.08587697148323059
[epoch = 0, idx = 5300]:   D_loss = 0.16993850469589233, 	 G_loss = 2.4118592739105225, 	 identity_loss = 0.073123499751091, 	 cycle_loss = 0.06829573214054108
[epoch = 1, idx = 0]:   D_loss = 0.2519295811653137, 	 G_loss = 2.6022677421569824, 	 identity_loss = 0.07938376069068909, 	 cycle_loss = 0.07570932060480118
[epoch = 1, idx = 100]:   D_loss = 0.30415016412734985, 	 G_loss = 2.0491199493408203, 	 identity_loss = 0.06486701220273972, 	 cycle_loss = 0.06404197216033936
[epoch = 1, idx = 200]:   D_loss = 0.23953774571418762, 	 G_loss = 3.4566380977630615, 	 identity_loss = 0.11204945296049118, 	 cycle_loss = 0.10928121209144592
[epoch = 1, idx = 300]:   D_loss = 

[epoch = 1, idx = 4800]:   D_loss = 3.953826155367324e-11, 	 G_loss = 2.544496536254883, 	 identity_loss = 0.02922828495502472, 	 cycle_loss = 0.03983639180660248
[epoch = 1, idx = 4900]:   D_loss = 3.2243896441741526e-10, 	 G_loss = 2.608935594558716, 	 identity_loss = 0.032082557678222656, 	 cycle_loss = 0.04485228657722473
[epoch = 1, idx = 5000]:   D_loss = 5.466191765890471e-08, 	 G_loss = 2.682040214538574, 	 identity_loss = 0.04165918752551079, 	 cycle_loss = 0.04740610718727112
[epoch = 1, idx = 5100]:   D_loss = 1.490142211013623e-12, 	 G_loss = 2.4574825763702393, 	 identity_loss = 0.026895184069871902, 	 cycle_loss = 0.0323006808757782
[epoch = 1, idx = 5200]:   D_loss = 4.186496256001343e-14, 	 G_loss = 2.647137403488159, 	 identity_loss = 0.03429233282804489, 	 cycle_loss = 0.04756759852170944
[epoch = 1, idx = 5300]:   D_loss = 9.485703865316992e-16, 	 G_loss = 2.569190502166748, 	 identity_loss = 0.03573327884078026, 	 cycle_loss = 0.03905240818858147
[epoch = 2, idx = 0

[epoch = 2, idx = 4400]:   D_loss = 1.133180958271447e-19, 	 G_loss = 2.592001438140869, 	 identity_loss = 0.025042224675416946, 	 cycle_loss = 0.04667901247739792
[epoch = 2, idx = 4500]:   D_loss = 1.8044940033586174e-15, 	 G_loss = 2.5676021575927734, 	 identity_loss = 0.03817848488688469, 	 cycle_loss = 0.03767097741365433
[epoch = 2, idx = 4600]:   D_loss = 5.515668064832611e-15, 	 G_loss = 2.650083065032959, 	 identity_loss = 0.036748118698596954, 	 cycle_loss = 0.046634264290332794
[epoch = 2, idx = 4700]:   D_loss = 6.082948581861037e-17, 	 G_loss = 2.5364110469818115, 	 identity_loss = 0.029112890362739563, 	 cycle_loss = 0.03908465802669525
[epoch = 2, idx = 4800]:   D_loss = 1.8931461909232894e-07, 	 G_loss = 2.7601962089538574, 	 identity_loss = 0.04142375290393829, 	 cycle_loss = 0.05530774965882301
[epoch = 2, idx = 4900]:   D_loss = 2.372799218494984e-16, 	 G_loss = 2.463388681411743, 	 identity_loss = 0.026204939931631088, 	 cycle_loss = 0.03323640674352646
[epoch = 2, 

[epoch = 3, idx = 4000]:   D_loss = 9.025422593528637e-21, 	 G_loss = 2.5186142921447754, 	 identity_loss = 0.025185476988554, 	 cycle_loss = 0.03926867991685867
[epoch = 3, idx = 4100]:   D_loss = 3.484934668152751e-18, 	 G_loss = 2.4548301696777344, 	 identity_loss = 0.02582745999097824, 	 cycle_loss = 0.032569289207458496
[epoch = 3, idx = 4200]:   D_loss = 1.7763578981914345e-15, 	 G_loss = 2.403223991394043, 	 identity_loss = 0.0270658228546381, 	 cycle_loss = 0.026789497584104538
[epoch = 3, idx = 4300]:   D_loss = 1.0761399532476798e-18, 	 G_loss = 2.484776258468628, 	 identity_loss = 0.024814557284116745, 	 cycle_loss = 0.03607034683227539
[epoch = 3, idx = 4400]:   D_loss = 1.7763670037956175e-13, 	 G_loss = 2.642080545425415, 	 identity_loss = 0.04148181155323982, 	 cycle_loss = 0.04346714913845062
[epoch = 3, idx = 4500]:   D_loss = 3.6651137341184655e-19, 	 G_loss = 2.388906717300415, 	 identity_loss = 0.021565470844507217, 	 cycle_loss = 0.028107916936278343
[epoch = 3, id

...

[epoch = 5, idx = 3200]:   D_loss = 1.848140390725561e-11, 	 G_loss = 2.4415743350982666, 	 identity_loss = 0.02396993152797222, 	 cycle_loss = 0.03217247501015663
[epoch = 5, idx = 3300]:   D_loss = 2.4065506773007912e-18, 	 G_loss = 2.496809482574463, 	 identity_loss = 0.025467056781053543, 	 cycle_loss = 0.036947425454854965
[epoch = 5, idx = 3400]:   D_loss = 6.821999557325196e-20, 	 G_loss = 2.538478374481201, 	 identity_loss = 0.03730405867099762, 	 cycle_loss = 0.035195812582969666
[epoch = 5, idx = 3500]:   D_loss = 1.7774586175063915e-15, 	 G_loss = 2.366274118423462, 	 identity_loss = 0.02106890082359314, 	 cycle_loss = 0.026092946529388428
[epoch = 5, idx = 3600]:   D_loss = 4.976536346264663e-20, 	 G_loss = 2.316882371902466, 	 identity_loss = 0.01981198601424694, 	 cycle_loss = 0.02178225666284561
[epoch = 5, idx = 3700]:   D_loss = 1.5177129266168558e-20, 	 G_loss = 2.395277500152588, 	 identity_loss = 0.024172967299818993, 	 cycle_loss = 0.02744126319885254
[epoch = 5, i

[epoch = 6, idx = 2800]:   D_loss = 1.8348561706681888e-17, 	 G_loss = 2.4566783905029297, 	 identity_loss = 0.02811143547296524, 	 cycle_loss = 0.03161214292049408
[epoch = 6, idx = 2900]:   D_loss = 2.814504475290823e-14, 	 G_loss = 2.4716899394989014, 	 identity_loss = 0.02809038758277893, 	 cycle_loss = 0.03312382847070694
[epoch = 6, idx = 3000]:   D_loss = 5.738979751637396e-17, 	 G_loss = 2.56754207611084, 	 identity_loss = 0.04318194091320038, 	 cycle_loss = 0.035163238644599915
[epoch = 6, idx = 3100]:   D_loss = 2.2125312317881878e-21, 	 G_loss = 2.3190102577209473, 	 identity_loss = 0.020144149661064148, 	 cycle_loss = 0.02182896062731743
[epoch = 6, idx = 3200]:   D_loss = 5.6659263985729735e-24, 	 G_loss = 2.515076160430908, 	 identity_loss = 0.03417978808283806, 	 cycle_loss = 0.034417711198329926
[epoch = 6, idx = 3300]:   D_loss = 1.0604256412594919e-19, 	 G_loss = 2.5276589393615723, 	 identity_loss = 0.02992788329720497, 	 cycle_loss = 0.03780195489525795
[epoch = 6, 

[epoch = 7, idx = 2400]:   D_loss = 1.7763568394002505e-15, 	 G_loss = 2.297860622406006, 	 identity_loss = 0.01699724979698658, 	 cycle_loss = 0.02128743939101696
[epoch = 7, idx = 2500]:   D_loss = 6.571306634888379e-23, 	 G_loss = 2.401017427444458, 	 identity_loss = 0.026030471548438072, 	 cycle_loss = 0.027086490765213966
[epoch = 7, idx = 2600]:   D_loss = 7.449621270482401e-25, 	 G_loss = 2.2800192832946777, 	 identity_loss = 0.016832631081342697, 	 cycle_loss = 0.019585616886615753
[epoch = 7, idx = 2700]:   D_loss = 1.481965613917409e-23, 	 G_loss = 2.4728376865386963, 	 identity_loss = 0.03347989171743393, 	 cycle_loss = 0.03054383024573326
[epoch = 7, idx = 2800]:   D_loss = 2.9617617490718964e-18, 	 G_loss = 2.4848780632019043, 	 identity_loss = 0.03341066837310791, 	 cycle_loss = 0.03178245201706886
[epoch = 7, idx = 2900]:   D_loss = 7.329144480606307e-23, 	 G_loss = 2.292576789855957, 	 identity_loss = 0.016968369483947754, 	 cycle_loss = 0.020773500204086304
[epoch = 7,

[epoch = 8, idx = 2000]:   D_loss = 1.250566835778395e-23, 	 G_loss = 2.3760640621185303, 	 identity_loss = 0.023029092699289322, 	 cycle_loss = 0.026091866195201874
[epoch = 8, idx = 2100]:   D_loss = 2.1847028991329603e-27, 	 G_loss = 2.380004644393921, 	 identity_loss = 0.019475135952234268, 	 cycle_loss = 0.028262900188565254
[epoch = 8, idx = 2200]:   D_loss = 8.940925770838919e-21, 	 G_loss = 2.5126895904541016, 	 identity_loss = 0.03273170813918114, 	 cycle_loss = 0.034903109073638916
[epoch = 8, idx = 2300]:   D_loss = 5.271731645231609e-19, 	 G_loss = 2.4189612865448, 	 identity_loss = 0.024482205510139465, 	 cycle_loss = 0.029655039310455322
[epoch = 8, idx = 2400]:   D_loss = 5.191731489162521e-09, 	 G_loss = 2.3856799602508545, 	 identity_loss = 0.02105637826025486, 	 cycle_loss = 0.02804998680949211
[epoch = 8, idx = 2500]:   D_loss = 2.6883344058579257e-20, 	 G_loss = 2.269907236099243, 	 identity_loss = 0.015650328248739243, 	 cycle_loss = 0.019165558740496635
[epoch = 8

[epoch = 9, idx = 1600]:   D_loss = 3.556167211365954e-24, 	 G_loss = 2.3956069946289062, 	 identity_loss = 0.025054436177015305, 	 cycle_loss = 0.02703346684575081
[epoch = 9, idx = 1700]:   D_loss = 1.7763623451144076e-15, 	 G_loss = 2.3100039958953857, 	 identity_loss = 0.018246982246637344, 	 cycle_loss = 0.021876920014619827
[epoch = 9, idx = 1800]:   D_loss = 7.105431592765738e-15, 	 G_loss = 2.350811004638672, 	 identity_loss = 0.01813683658838272, 	 cycle_loss = 0.026012685149908066
[epoch = 9, idx = 1900]:   D_loss = 4.0856237121765504e-13, 	 G_loss = 2.3588614463806152, 	 identity_loss = 0.022113868966698647, 	 cycle_loss = 0.024829227477312088
[epoch = 9, idx = 2000]:   D_loss = 1.1794639624915664e-23, 	 G_loss = 2.399106502532959, 	 identity_loss = 0.02384028024971485, 	 cycle_loss = 0.027990510687232018
[epoch = 9, idx = 2100]:   D_loss = 1.8142254652709038e-22, 	 G_loss = 2.5271480083465576, 	 identity_loss = 0.03849237412214279, 	 cycle_loss = 0.03346859663724899
[epoch 